In [1]:
import pandasql as ps 
import pandas as pd
import psycopg2 as pc

### Carrega os arquivos csv preparados em preprocessamento.ipynb

In [2]:
campeoes = pd.read_csv("campeoes.csv")
campeoes = campeoes.set_index('id_campeoes')

jogadores = pd.read_csv("jogadores.csv")
jogadores = jogadores.set_index('id_jogador')

times = pd.read_csv("times.csv")
times = times.set_index('id_time')

pre_jogo = pd.read_csv("pre_jogo.csv")
pre_jogo = pre_jogo.set_index('gameid')

peb = pd.read_csv('picks_e_bans.csv')
peb = peb.drop(columns='Unnamed: 0').set_index('gameid')

### Cria a base de dados

In [4]:
conn = pc.connect(
    host = "localhost", 
    database = "postgres",  
    user = "postgres", 
    password = "senha"
)
cur = conn.cursor()

comando = '''
CREATE SCHEMA cblol;
'''
cur.execute(comando)
conn.commit()
cur.close()


In [5]:
cur = conn.cursor()

comando = '''

CREATE TABLE cblol.campeoes (
  id_campeao SERIAL PRIMARY KEY,
  campeao varchar(30)
);

CREATE TABLE cblol.jogadores (
  id_jogador SERIAL PRIMARY KEY,
  jogador varchar(40)
);

CREATE TABLE cblol.times (
  id_time SERIAL PRIMARY KEY,
  time varchar(40)
);

CREATE TABLE cblol.jogos (
  id_jogo char(30) NOT NULL PRIMARY KEY,
  playoffs INTEGER,
  jogo integer,
  patch varchar(10),
  top_blue integer,
  jng_blue integer,
  mid_blue integer,
  bot_blue integer,
  sup_blue integer,
  teamid_blue integer,
  top_red integer,
  jng_red integer,
  mid_red integer,
  bot_red integer,
  sup_red integer,
  teamid_red integer,
  result integer,

  FOREIGN KEY (top_blue) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (jng_blue) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (mid_blue) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (bot_blue) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (sup_blue) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (teamid_blue) REFERENCES cblol.times (id_time),

  FOREIGN KEY (top_red) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (jng_red) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (mid_red) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (bot_red) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (sup_red) REFERENCES cblol.jogadores (id_jogador),
  FOREIGN KEY (teamid_red) REFERENCES cblol.times (id_time)

);

CREATE TABLE cblol.pickban (
  cod_jogo VARCHAR(30),
  ban_blue1 INTEGER,
  ban_blue2 INTEGER,
  ban_blue3 INTEGER,
  ban_blue4 INTEGER,
  ban_blue5 INTEGER,
  top_blue INTEGER,
  jng_blue INTEGER,
  mid_blue INTEGER,
  bot_blue INTEGER,
  sup_blue INTEGER,
  ban_red1 INTEGER,
  ban_red2 INTEGER,
  ban_red3 INTEGER,
  ban_red4 INTEGER,
  ban_red5 INTEGER,
  top_red INTEGER,
  jng_red INTEGER,
  mid_red INTEGER,
  bot_red INTEGER,
  sup_red INTEGER,
  FOREIGN KEY (cod_jogo) REFERENCES cblol.jogos (id_jogo)
);

'''
cur.execute(comando)
conn.commit()
cur.close()

In [6]:
### Tratamento de strings que contem apostrofo
for i in range(campeoes.shape[0]):
    campeoes.nome[i] = str(campeoes.nome[i]).replace("'","")

### Populando a base de dados

In [7]:
for i,j in list(zip(campeoes.index,campeoes.values)):
    cur = conn.cursor()
    query=f'''INSERT INTO cblol.campeoes (id_campeao,campeao) VALUES ({int(i)},'{j[0]}')'''
    cur.execute(query)
    cur.close()

for i,j in list(zip(jogadores.index,jogadores.values)):
    cur = conn.cursor()
    query=f'''INSERT INTO cblol.jogadores (id_jogador,jogador) VALUES ({int(i)},'{j[0]}')'''
    cur.execute(query)
    cur.close()

for i,j in list(zip(times.index,times.values)):
    cur = conn.cursor()
    query=f'''INSERT INTO cblol.times (id_time,time) VALUES ({int(i)},'{j[0]}')'''
    cur.execute(query)
    cur.close()

for i,j in list(zip(pre_jogo.index,pre_jogo.values)):
    cur = conn.cursor()
    query=f'''INSERT INTO cblol.jogos (id_jogo,playoffs, jogo, patch, top_blue, jng_blue, mid_blue, bot_blue, sup_blue, teamid_blue, top_red, jng_red, mid_red, bot_red, sup_red, teamid_red, result) 
        VALUES ('{i}',{j[0]},{j[1]},'{j[2]}',{j[3]},{j[4]},{j[5]},{j[6]},{j[7]},{j[8]},{j[9]},{j[10]},{j[11]},{j[12]},{j[13]},{j[14]},{j[15]})'''
    cur.execute(query)
    cur.close()

for i,j in list(zip(peb.index,peb.values)):
    cur = conn.cursor()
    query=f'''INSERT INTO cblol.pickban (cod_jogo,ban_blue1, ban_blue2, ban_blue3, ban_blue4, ban_blue5, top_blue, jng_blue, mid_blue, bot_blue, sup_blue, ban_red1, ban_red2, ban_red3, ban_red4, ban_red5, top_red, jng_red, mid_red, bot_red, sup_red) 
        VALUES ('{i}',{j[0]},{j[1]},{j[2]},{j[3]},{j[4]},{j[5]},{j[6]},{j[7]},{j[8]},{j[9]},{j[10]},{j[11]},{j[12]},{j[13]},{j[14]},{j[15]},{j[16]},{j[17]},{j[18]},{j[19]})'''
    cur.execute(query)
    cur.close()

    
conn.commit()

# Algumas perguntas

In [8]:
print("Campeoes mais banidos de 2022 \n")

com = """
SELECT * FROM (
	SELECT
		c.campeao, COUNT(c.campeao) AS n_bans
	FROM cblol.campeoes  c
	INNER JOIN 
		(SELECT p.ban_blue1 AS b1 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_blue2 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_blue3 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_blue4 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_blue5 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_red1 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_red2 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_red3 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_red4 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.ban_red5 FROM cblol.pickban p
    ) s ON s.b1 = c.id_campeao--necessário nomear o union para funcinonar
GROUP BY c.campeao) t
ORDER BY t.n_bans DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

Campeoes mais banidos de 2022 



C:\Users\antph\AppData\Local\Temp\ipykernel_9716\3931641972.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cod = pd.read_sql_query(com, con = conn)


,campeao,n_bans
0,Zeri,122
1,Gwen,117
2,Caitlyn,85
3,Lee Sin,73
4,Ahri,68
5,Vex,64
6,Jayce,62
7,Lucian,61
8,Kalista,61
9,Twisted Fate,60


In [11]:
print("Campeoes mais escolhidos do lado azul de 2022 \n")

com = """
SELECT * FROM (
	SELECT c.campeao, COUNT(c.campeao) AS n_picks
	FROM cblol.campeoes  c
	INNER JOIN 
		(SELECT p.top_blue AS b1 FROM cblol.pickban p
		 UNION ALL
		 SELECT p.jng_blue FROM cblol.pickban p
		 UNION ALL
		 SELECT p.mid_blue FROM cblol.pickban p
		 UNION ALL
		 SELECT p.bot_blue FROM cblol.pickban p
		 UNION ALL
		 SELECT p.sup_blue FROM cblol.pickban p
    ) s ON s.b1 = c.id_campeao--necessário nomear o union para funcinonar
GROUP BY c.campeao) t
ORDER BY t.n_picks DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

Campeoes mais escolhidos do lado azul de 2022 



C:\Users\antph\AppData\Local\Temp\ipykernel_9716\1211567140.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cod = pd.read_sql_query(com, con = conn)


,campeao,n_picks
0,Jinx,61
1,Nautilus,48
2,Ahri,48
3,Viego,45
4,Gnar,42
5,Zeri,41
6,Leona,38
7,Lee Sin,35
8,Aphelios,32
9,Gwen,29


In [13]:
print("Time que mais venceu partidas únicas em 2022")

com = """
SELECT * FROM (
	SELECT t.time,count(t.time) AS vitorias
	FROM cblol.times t
	INNER JOIN 
		(SELECT j.teamid_blue AS vitorias FROM cblol.jogos j
		 WHERE j.result = 1
		 UNION ALL
		 SELECT j.teamid_red AS vitorias FROM cblol.jogos j
		 WHERE j.result = 0
    ) s ON s.vitorias = t.id_time--necessário nomear o union para funcinonar
GROUP BY t.time) b
ORDER BY b.vitorias DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

Time que mais venceu partidas únicas em 2022


C:\Users\antph\AppData\Local\Temp\ipykernel_9716\1977269702.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cod = pd.read_sql_query(com, con = conn)


,time,vitorias
0,RED Canids,42
1,paiN Gaming,36
2,FURIA,34
3,LOUD,33
4,KaBuM! e-Sports,32
5,Liberty,23
6,Netshoes Miners,10
7,Miners,8
8,Rensga eSports,7
9,INTZ,7


In [14]:
conn.close()